In [14]:
## imports
import pandas as pd
import numpy as np
import plotnine
from plotnine import *
import random

## print multiple things from same cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from datetime import datetime, timedelta

## Load data

In [16]:
## load data on 2020 crimes in DC
df = dc_crim_2020 = pd.read_csv("https://opendata.arcgis.com/datasets/f516e0dd7b614b088ad781b0c4002331_2.csv")

## create report_dt column
df['report_dt'] = pd.to_datetime(df.REPORT_DAT)

## Warm-up Demo

In [21]:
%%time
for i in range(df.shape[0]):
    r = df.iloc[i]
    r.X + r.Y

CPU times: user 1.32 s, sys: 7.35 ms, total: 1.33 s
Wall time: 1.33 s


In [22]:
%%time
for i,r in df.iterrows():
    r.X + r.Y

CPU times: user 847 ms, sys: 11.5 ms, total: 859 ms
Wall time: 857 ms


In [43]:
df.shape

(27932, 26)

In [45]:
%%time
df.apply(lambda r: r.X + r.Y, axis = 1)

CPU times: user 274 ms, sys: 8.05 ms, total: 282 ms
Wall time: 278 ms


0       -38.048603
1       -38.115721
2       -38.097421
3       -38.050871
4       -38.051294
           ...    
27927   -38.139043
27928   -38.055147
27929   -38.088756
27930   -38.110882
27931   -38.106803
Length: 27932, dtype: float64

In [24]:
%%time
## Super fast, but only works with built-in numpy functions.
df.X + df.Y

CPU times: user 1.3 ms, sys: 67 µs, total: 1.37 ms
Wall time: 1.52 ms


0       -38.048603
1       -38.115721
2       -38.097421
3       -38.050871
4       -38.051294
           ...    
27927   -38.139043
27928   -38.055147
27929   -38.088756
27930   -38.110882
27931   -38.106803
Length: 27932, dtype: float64

# Practice

In [25]:
## define crimes to look for and crimes to look within
## CCN is Central Complaint Number: https://go.mpdconline.com/GO/GO_401_01.pdf
CCN_examples = ['20165648', '20123250']
C_Tar = C_Target = crimes_lookfor = df[df.CCN.astype(str).isin(CCN_examples)][['CCN', 'WARD', 'OFFENSE', 'report_dt']]
C_Oth = C_Other  = other_crimes = df[~df.CCN.astype(str).isin(CCN_examples)]

## print crimes_lookfor
C_Tar.head()
# other_crimes.head()

,CCN,WARD,OFFENSE,report_dt
12639,20123250,2,MOTOR VEHICLE THEFT,2020-08-29 05:00:25+00:00
19204,20165648,6,MOTOR VEHICLE THEFT,2020-11-20 02:25:50+00:00


**Task**: we have two crimes we want to look for. We want to look in the remaining crime reports for crime reports that are:

- Located in the same ward as the two focal crimes
- Reported at the same time as the focal crime or up to 1000 minutes later (changed from slides which stated 20 mins since crime ids changed since last time so this long bandwidth helps us find matches!)

Solutions compare two ways to solve:

- Using a for loop
- Using a function

## 1. Loop approach

In [7]:
## create empty container to store results 
store_matches = {}

## loop through two example crimes
for i in range(C_Tar.shape[0]): # same as 
    
    ## extract row
    r = one_row = C_Tar.iloc[i]

    ## first, subset to crimes in same ward
    same_wards = C_Oth[C_Oth.WARD == r.WARD]
    
    ## second, with those same-ward crimes, construct indicator for reported within 20 minutes
    ## (interpreting as after but could do either)
    ### substep: get time cutoff
    CUTOFF = r.report_dt +  timedelta(minutes=1200)
    
    ### substep: use that to subset
    same_wards_sametime = same_wards[(same_wards.report_dt >= r.report_dt) & 
                                    (same_wards.report_dt <= CUTOFF)].copy()
    
    ## third, store the results
    store_matches[str(one_row.CCN)] = same_wards_sametime
    
## finally, concatenate results into one df
all_matches = pd.concat(store_matches)
all_matches.head()

X          Y       CCN              REPORT_DAT  \
20123250 6048  -77.021929  38.899129  20123419  2020/08/29 17:15:19+00   
         11308 -77.050528  38.913354  20123422  2020/08/29 16:45:57+00   
         13946 -77.038491  38.913727  20401318  2020/08/29 14:29:59+00   
         16387 -77.040091  38.909646  20123389  2020/08/29 16:05:18+00   
         23620 -77.039824  38.905656  20123507  2020/08/29 22:04:46+00   

                  SHIFT  METHOD              OFFENSE  \
20123250 6048       DAY  OTHERS          THEFT/OTHER   
         11308      DAY  OTHERS         THEFT F/AUTO   
         13946      DAY  OTHERS          THEFT/OTHER   
         16387      DAY  OTHERS         THEFT F/AUTO   
         23620  EVENING  OTHERS  MOTOR VEHICLE THEFT   

                                                BLOCK    XBLOCK    YBLOCK  \
20123250 6048        700 - 799 BLOCK OF 7TH STREET NW  398098.0  136808.0   
         11308  2200 - 2399 BLOCK OF DECATUR PLACE NW  395618.0  138388.0   
         13946    1724 - 1799 BLOCK OF 17TH STREET NW  396662.0  138429.0   
         16387       1700 - 1799 BLOCK OF P STREET NW  396523.0  137976.0   
         23620       1700 - 1779 BLOCK OF M STREET NW  396546.0  137533.0   

                ...  CENSUS_TRACT VOTING_PRECINCT   LATITUDE  LONGITUDE  \
20123250 6048   ...        5801.0    Precinct 129  38.899121 -77.021926   
         11308  ...        4100.0     Precinct 13  38.913346 -77.050526   
         13946  ...        5302.0     Precinct 15  38.913720 -77.038489   
         16387  ...        5303.0     Precinct 15  38.909638 -77.040089   
         23620  ...       10700.0     Precinct 17  38.905648 -77.039822   

                            BID              START_DATE  \
20123250 6048          DOWNTOWN  2020/08/29 16:05:40+00   
         11308              NaN  2020/08/26 22:00:29+00   
         13946              NaN  2020/08/28 20:55:00+00   
         16387              NaN  2020/08/28 22:00:23+00   
         23620  GOLDEN TRIANGLE  2020/08/27 19:01:24+00   

                              END_DATE   OBJECTID  OCTO_RECORD_ID  \
20123250 6048   2020/08/29 16:08:33+00  500137941             NaN   
         11308  2020/08/27 12:00:51+00  500147812             NaN   
         13946  2020/08/28 21:05:00+00  500157540             NaN   
         16387  2020/08/29 08:00:27+00  500183867             NaN   
         23620  2020/08/29 19:00:05+00  500366738             NaN   

                               report_dt  
20123250 6048  2020-08-29 17:15:19+00:00  
         11308 2020-08-29 16:45:57+00:00  
         13946 2020-08-29 14:29:59+00:00  
         16387 2020-08-29 16:05:18+00:00  
         23620 2020-08-29 22:04:46+00:00  

[5 rows x 26 columns]

# 1.5 Iterrow Approach

In [8]:
## create empty container to store results 
store_matches = {}

## loop through two example crimes
for i, r in C_Tar.iterrows(): # same as 

    ## subset to crimes in same ward
    same_wards = C_Oth[C_Oth.WARD == r.WARD]
    
    ## second, with those same-ward crimes, construct indicator for reported within 20 minutes
    ## (interpreting as after but could do either)
    ### substep: get time cutoff
    CUTOFF = r.report_dt +  timedelta(minutes=1200)
    
    ### substep: use that to subset
    same_wards_sametime = same_wards[(same_wards.report_dt >= r.report_dt) & 
                                    (same_wards.report_dt <= CUTOFF)].copy()
    
    ## third, store the results
    store_matches[str(one_row.CCN)] = same_wards_sametime
    
## finally, concatenate results into one df
all_matches = pd.concat(store_matches)
all_matches.head()

X          Y       CCN              REPORT_DAT  \
20165648 491   -77.015554  38.899950  20166039  2020/11/20 22:07:10+00   
         10824 -76.997328  38.885141  20165798  2020/11/20 12:46:32+00   
         14887 -77.005894  38.905167  20165986  2020/11/20 22:17:27+00   
         16277 -76.999518  38.891483  20165709  2020/11/20 04:27:36+00   
         16280 -77.001316  38.898915  20165932  2020/11/20 18:56:18+00   

                   SHIFT  METHOD              OFFENSE  \
20165648 491     EVENING  OTHERS          THEFT/OTHER   
         10824       DAY  OTHERS          THEFT/OTHER   
         14887   EVENING  OTHERS  MOTOR VEHICLE THEFT   
         16277  MIDNIGHT  OTHERS  MOTOR VEHICLE THEFT   
         16280       DAY  OTHERS         THEFT F/AUTO   

                                                     BLOCK    XBLOCK  \
20165648 491    300 - 363 BLOCK OF MASSACHUSETTS AVENUE NW  398651.0   
         10824   600 - 669 BLOCK OF PENNSYLVANIA AVENUE SE  400232.0   
         14887          1151 - 1199 BLOCK OF 1ST STREET NE  399489.0   
         16277            100 - 199 BLOCK OF 5TH STREET NE  400042.0   
         16280              300 - 399 BLOCK OF G STREET NE  399886.0   

                  YBLOCK  ...  CENSUS_TRACT VOTING_PRECINCT   LATITUDE  \
20165648 491    136899.0  ...        5900.0    Precinct 143  38.899942   
         10824  135255.0  ...        6500.0     Precinct 89  38.885133   
         14887  137478.0  ...       10603.0    Precinct 144  38.905159   
         16277  135959.0  ...        8200.0     Precinct 89  38.891475   
         16280  136784.0  ...        8301.0     Precinct 83  38.898907   

                LONGITUDE           BID              START_DATE  \
20165648 491   -77.015552      DOWNTOWN  2020/11/20 17:30:16+00   
         10824 -76.997326  CAPITOL HILL  2020/11/19 23:43:15+00   
         14887 -77.005891          NOMA  2020/11/20 20:15:26+00   
         16277 -76.999516           NaN  2020/11/20 03:02:27+00   
         16280 -77.001314           NaN  2020/11/20 15:30:02+00   

                              END_DATE   OBJECTID  OCTO_RECORD_ID  \
20165648 491    2020/11/20 22:08:28+00  499865401             NaN   
         10824                     NaN  500146815             NaN   
         14887  2020/11/20 21:46:24+00  500163163             NaN   
         16277                     NaN  500182525             NaN   
         16280  2020/11/20 18:25:35+00  500182528             NaN   

                               report_dt  
20165648 491   2020-11-20 22:07:10+00:00  
         10824 2020-11-20 12:46:32+00:00  
         14887 2020-11-20 22:17:27+00:00  
         16277 2020-11-20 04:27:36+00:00  
         16280 2020-11-20 18:56:18+00:00  

[5 rows x 26 columns]

## 2. Function approach

Practice rewriting the above loop as a function

In [41]:
C_Tar

,CCN,WARD,OFFENSE,report_dt
12639,20123250,2,MOTOR VEHICLE THEFT,2020-08-29 05:00:25+00:00
19204,20165648,6,MOTOR VEHICLE THEFT,2020-11-20 02:25:50+00:00


### 2.1 define the function

In [ ]:
store_matches_2 = {}

def find_related_crimes(r): # imagine the function taking in one row as its sole variable
    # Your code here

In [52]:
store_matches_2 = {}

def new_crimes(r):
    same_wards = C_Oth[C_Oth.WARD == r.WARD]

    CUTOFF = r.report_dt +  timedelta(minutes=1200)

    same_wards_sametime = same_wards[(same_wards.report_dt >= r.report_dt) &
                                    (same_wards.report_dt <= CUTOFF)].copy()
    store_matches_2[str(r.CCN)] = same_wards_sametime

### 2.2 apply it to one of the focal crimes

In [63]:
first_row = C_Tar.iloc[0]
new_crimes( first_row )
store_matches_2

{'20123250':                X          Y       CCN              REPORT_DAT    SHIFT  \
 6048  -77.021929  38.899129  20123419  2020/08/29 17:15:19+00      DAY   
 11308 -77.050528  38.913354  20123422  2020/08/29 16:45:57+00      DAY   
 13946 -77.038491  38.913727  20401318  2020/08/29 14:29:59+00      DAY   
 16387 -77.040091  38.909646  20123389  2020/08/29 16:05:18+00      DAY   
 23620 -77.039824  38.905656  20123507  2020/08/29 22:04:46+00  EVENING   
 27694 -77.027565  38.897353  20123609  2020/08/30 00:05:52+00  EVENING   
 
        METHOD              OFFENSE                                  BLOCK  \
 6048   OTHERS          THEFT/OTHER       700 - 799 BLOCK OF 7TH STREET NW   
 11308  OTHERS         THEFT F/AUTO  2200 - 2399 BLOCK OF DECATUR PLACE NW   
 13946  OTHERS          THEFT/OTHER    1724 - 1799 BLOCK OF 17TH STREET NW   
 16387  OTHERS         THEFT F/AUTO       1700 - 1799 BLOCK OF P STREET NW   
 23620  OTHERS  MOTOR VEHICLE THEFT       1700 - 1779 BLOCK OF M STREET

### 2.3 Use apply to cover all the other focal crimes

In [55]:
C_Tar.apply(new_crimes, axis=1)
pd.concat( store_matches_2 )

12639    None
19204    None
dtype: object

X          Y       CCN              REPORT_DAT  \
20123250 6048  -77.021929  38.899129  20123419  2020/08/29 17:15:19+00   
         11308 -77.050528  38.913354  20123422  2020/08/29 16:45:57+00   
         13946 -77.038491  38.913727  20401318  2020/08/29 14:29:59+00   
         16387 -77.040091  38.909646  20123389  2020/08/29 16:05:18+00   
         23620 -77.039824  38.905656  20123507  2020/08/29 22:04:46+00   
         27694 -77.027565  38.897353  20123609  2020/08/30 00:05:52+00   
20165648 491   -77.015554  38.899950  20166039  2020/11/20 22:07:10+00   
         10824 -76.997328  38.885141  20165798  2020/11/20 12:46:32+00   
         14887 -77.005894  38.905167  20165986  2020/11/20 22:17:27+00   
         16277 -76.999518  38.891483  20165709  2020/11/20 04:27:36+00   
         16280 -77.001316  38.898915  20165932  2020/11/20 18:56:18+00   
         20054 -76.997316  38.904969  20165805  2020/11/20 15:06:04+00   
         21231 -76.994365  38.900203  20165859  2020/11/20 15:37:59+00   
         27177 -76.997316  38.904969  20165803  2020/11/20 14:45:06+00   

                   SHIFT  METHOD              OFFENSE  \
20123250 6048        DAY  OTHERS          THEFT/OTHER   
         11308       DAY  OTHERS         THEFT F/AUTO   
         13946       DAY  OTHERS          THEFT/OTHER   
         16387       DAY  OTHERS         THEFT F/AUTO   
         23620   EVENING  OTHERS  MOTOR VEHICLE THEFT   
         27694   EVENING  OTHERS  MOTOR VEHICLE THEFT   
20165648 491     EVENING  OTHERS          THEFT/OTHER   
         10824       DAY  OTHERS          THEFT/OTHER   
         14887   EVENING  OTHERS  MOTOR VEHICLE THEFT   
         16277  MIDNIGHT  OTHERS  MOTOR VEHICLE THEFT   
         16280       DAY  OTHERS         THEFT F/AUTO   
         20054       DAY  OTHERS         THEFT F/AUTO   
         21231       DAY  OTHERS          THEFT/OTHER   
         27177       DAY  OTHERS         THEFT F/AUTO   

                                                     BLOCK    XBLOCK  \
20123250 6048             700 - 799 BLOCK OF 7TH STREET NW  398098.0   
         11308       2200 - 2399 BLOCK OF DECATUR PLACE NW  395618.0   
         13946         1724 - 1799 BLOCK OF 17TH STREET NW  396662.0   
         16387            1700 - 1799 BLOCK OF P STREET NW  396523.0   
         23620            1700 - 1779 BLOCK OF M STREET NW  396546.0   
         27694            1100 - 1199 BLOCK OF F STREET NW  397609.0   
20165648 491    300 - 363 BLOCK OF MASSACHUSETTS AVENUE NW  398651.0   
         10824   600 - 669 BLOCK OF PENNSYLVANIA AVENUE SE  400232.0   
         14887          1151 - 1199 BLOCK OF 1ST STREET NE  399489.0   
         16277            100 - 199 BLOCK OF 5TH STREET NE  400042.0   
         16280              300 - 399 BLOCK OF G STREET NE  399886.0   
         20054         600 - 699 BLOCK OF ORLEANS PLACE NE  400233.0   
         21231              800 - 899 BLOCK OF H STREET NE  400489.0   
         27177         600 - 699 BLOCK OF ORLEANS PLACE NE  400233.0   

                  YBLOCK  ...  CENSUS_TRACT VOTING_PRECINCT   LATITUDE  \
20123250 6048   136808.0  ...        5801.0    Precinct 129  38.899121   
         11308  138388.0  ...        4100.0     Precinct 13  38.913346   
         13946  138429.0  ...        5302.0     Precinct 15  38.913720   
         16387  137976.0  ...        5303.0     Precinct 15  38.909638   
         23620  137533.0  ...       10700.0     Precinct 17  38.905648   
         27694  136611.0  ...        5802.0    Precinct 129  38.897346   
20165648 491    136899.0  ...        5900.0    Precinct 143  38.899942   
         10824  135255.0  ...        6500.0     Precinct 89  38.885133   
         14887  137478.0  ...       10603.0    Precinct 144  38.905159   
         16277  135959.0  ...        8200.0     Precinct 89  38.891475   
         16280  136784.0  ...        8301.0     Precinct 83  38.898907   
         20054  137456.0  ...       10602.0     Precinct 83  38.904961   
         21231

In [58]:
pd.concat( store_matches_2.values() )

,X,Y,CCN,REPORT_DAT,SHIFT,METHOD,OFFENSE,BLOCK,XBLOCK,YBLOCK,...,CENSUS_TRACT,VOTING_PRECINCT,LATITUDE,LONGITUDE,BID,START_DATE,END_DATE,OBJECTID,OCTO_RECORD_ID,report_dt
6048,-77.021929,38.899129,20123419,2020/08/29 17:15:19+00,DAY,OTHERS,THEFT/OTHER,700 - 799 BLOCK OF 7TH STREET NW,398098.0,136808.0,...,5801.0,Precinct 129,38.899121,-77.021926,DOWNTOWN,2020/08/29 16:05:40+00,2020/08/29 16:08:33+00,500137941,NaN,2020-08-29 17:15:19+00:00
11308,-77.050528,38.913354,20123422,2020/08/29 16:45:57+00,DAY,OTHERS,THEFT F/AUTO,2200 - 2399 BLOCK OF DECATUR PLACE NW,395618.0,138388.0,...,4100.0,Precinct 13,38.913346,-77.050526,NaN,2020/08/26 22:00:29+00,2020/08/27 12:00:51+00,500147812,NaN,2020-08-29 16:45:57+00:00
13946,-77.038491,38.913727,20401318,2020/08/29 14:29:59+00,DAY,OTHERS,THEFT/OTHER,1724 - 1799 BLOCK OF 17TH STREET NW,396662.0,138429.0,...,5302.0,Precinct 15,38.913720,-77.038489,NaN,2020/08/28 20:55:00+00,2020/08/28 21:05:00+00,500157540,NaN,2020-08-29 14:29:59+00:00
16387,-77.040091,38.909646,20123389,2020/08/29 16:05:18+00,DAY,OTHERS,THEFT F/AUTO,1700 - 1799 BLOCK OF P STREET NW,396523.0,137976.0,...,5303.0,Precinct 15,38.909638,-77.040089,NaN,2020/08/28 22:00:23+00,2020/08/29 08:00:27+00,500183867,NaN,2020-08-29 16:05:18+00:00
23620,-77.039824,38.905656,20123507,2020/08/29 22:04:46+00,EVENING,OTHERS,MOTOR VEHICLE THEFT,1700 - 1779 BLOCK OF M STREET NW,396546.0,137533.0,...,10700.0,Precinct 17,38.905648,-77.039822,GOLDEN TRIANGLE,2020/08/27 19:01:24+00,2020/08/29 19:00:05+00,500366738,NaN,2020-08-29 22:04:46+00:00
27694,-77.027565,38.897353,20123609,2020/08/30 00:05:52+00,EVENING,OTHERS,MOTOR VEHICLE THEFT,1100 - 1199 BLOCK OF F STREET NW,397609.0,136611.0,...,5802.0,Precinct 129,38.897346,-77.027563,DOWNTOWN,2020/08/29 23:08:57+00,NaN,500400836,NaN,2020-08-30 00:05:52+00:00
491,-77.015554,38.899950,20166039,2020/11/20 22:07:10+00,EVENING,OTHERS,THEFT/OTHER,300 - 363 BLOCK OF MASSACHUSETTS AVENUE NW,398651.0,136899.0,...,5900.0,Precinct 143,38.899942,-77.015552,DOWNTOWN,2020/11/20 17:30:16+00,2020/11/20 22:08:28+00,499865401,NaN,2020-11-20 22:07:10+00:00
10824,-76.997328,38.885141,20165798,2020/11/20 12:46:32+00,DAY,OTHERS,THEFT/OTHER,600 - 669 BLOCK OF PENNSYLVANIA AVENUE SE,400232.0,135255.0,...,6500.0,Precinct 89,38.885133,-76.997326,CAPITOL HILL,2020/11/19 23:43:15+00,NaN,500146815,NaN,2020-11-20 12:46:32+00:00
14887,-77.005894,38.905167,20165986,2020/11/20 22:17:27+00,EVENING,OTHERS,MOTOR VEHICLE THEFT,1151 - 1199 BLOCK OF 1ST STREET NE,399489.0,137478.0,...,10603.0,Precinct 144,38.905159,-77.005891,NOMA,2020/11/20 20:15:26+00,2020/11/20 21:46:24+00,500163163,NaN,2020-11-20 22:17:27+00:00
16277,-76.999518,38.891483,20165709,2020/11/20 04:27:36+00,MIDNIGHT,OTHERS,MOTOR VEHICLE THEFT,100 - 199 BLOCK OF 5TH STREET NE,400042.0,135959.0,...,8200.0,Precinct 89,38.891475,-76.999516,NaN,2020/11/20 03:02:27+00,NaN,500182525,NaN,2020-11-20 04:27:36+00:00


In [67]:
def find_related_crimes(r):
    same_wards = C_Oth[C_Oth.WARD == r.WARD]
    CUTOFF = r.report_dt +  timedelta(minutes=1200)
    same_wards_sametime = same_wards[(same_wards.report_dt >= r.report_dt) &
                                    (same_wards.report_dt <= CUTOFF)].copy()
    store_matches_2[str(r.CCN)] = same_wards_sametime
    return same_wards_sametime

# store_matches_2 = {}
# for i,r in (C_Tar.iterrows()):
# #     print(find_related_crimes(r))
#     print(len(find_related_crimes(r)))